In [46]:
"""
# Grouped Median Income & Median Age Tabulation for SANDAG's Demographic data
Last Updated: 12/22/22
@author: Michael Ma
"""

## Median Income

In [48]:
import pandas as pd
import sqlalchemy as sql
import pymssql
import numpy as np

# Demographic Data from SQL Server
# Get the demographic data from the database (Household Income)
ddam = sql.create_engine('mssql+pymssql://xxxx/')
income = pd.read_sql(
"""   
SELECT d.tract, g.income_group_id ,g.name AS income, SUM(i.households) AS households
FROM demographic_warehouse.fact.household_income i 
JOIN demographic_warehouse.dim.income_group g ON i.income_group_id = g.income_group_id
JOIN demographic_warehouse.dim.mgra_denormalize d ON i.mgra_id = d.mgra_id
WHERE datasource_id = 45 AND yr_id = 2020
GROUP BY d.tract, g.name, g.income_group_id
ORDER BY d.tract, g.income_group_id
""",con=ddam.connect())
# display(income)


def income_group_lower_bound(x):
    if x == 'Less than $15,000':
        return 0
    if x == '$15,000 to $29,999':
        return 15000
    if x == '$30,000 to $44,999':
        return 30000
    if x == '$45,000 to $59,999':
        return 45000
    if x == '$60,000 to $74,999':
        return 60000
    if x == '$75,000 to $99,999':
        return 75000
    if x == '$100,000 to $124,999':
        return 100000
    if x == '$125,000 to $149,999':
        return 125000
    if x == '$150,000 to $199,999':
        return 150000
    if x == '$200,000 or more':
        return 200000
    
income['lower_bound'] = income['income'].apply(income_group_lower_bound)


def income_group_upper_bound(x):
    if x == 'Less than $15,000':
        return 15000
    if x == '$15,000 to $29,999':
        return 29999
    if x == '$30,000 to $44,999':
        return 44999
    if x == '$45,000 to $59,999':
        return 59999
    if x == '$60,000 to $74,999':
        return 74999
    if x == '$75,000 to $99,999':
        return 99999
    if x == '$100,000 to $124,999':
        return 124999
    if x == '$125,000 to $149,999':
        return 149999
    if x == '$150,000 to $199,999':
        return 199999
    if x == '$200,000 or more':
        return 200000
    
income['upper_bound'] = income['income'].apply(income_group_upper_bound)


# # Median Income for Grouped Data (Source: https://www.statology.org/median-of-grouped-data/)
# '''
# L: Lower limit of median class
# W: Width of median class
# N: Total Frequency
# C: Cumulative frequency up to median class
# F: Frequency of median class

# Median = L + W[(N/2 - C) / F]
# '''

# # Median Income Function (For differnt geo)
def MedianIncome(Table, Geo):
    newtable = []

    for y in Table[Geo].unique():
        x = Table[Table[Geo] == y]
        
        # If the sum of households is 0, then assign -1 as the median and append to the table
        if x['households'].sum() == 0:
            newtable.append([y, -1])
            continue
        x['cumulative_households'] = x['households'].cumsum(axis=0)
        
        # Check the dataframe
        # print(y)
        # print(x[x['cumulative_households'] > x['households'].sum()/2])

        x_median_income_class = x[x['cumulative_households'] > x['households'].sum()/2].iloc[[0]]
        x_precedingcumulative = x[x['cumulative_households'] < x['households'].sum()/2].iloc[[-1]]
        
        lower_limit = x_median_income_class.iloc[0]['lower_bound']
        width = (x_median_income_class.iloc[0]['upper_bound'] - x_median_income_class.iloc[0]['lower_bound'])
        N = x['households'].sum()
        C = x_precedingcumulative.iloc[0]['cumulative_households']
        F = x_median_income_class.iloc[0]['households']
        
        newtable.append([y, (lower_limit + (width * ((N/2 - C) / F)).round())])

    return pd.DataFrame(newtable, columns=['Geography','Median_Income'])

In [ ]:
MedianIncome(income, 'tract')

## Median Age

In [24]:
import pandas as pd
import sqlalchemy as sql
import pymssql
import numpy as np


# Demographic Data from SQL Server
# Get the demographic data from the database (Population by Age)
ddam = sql.create_engine('mssql+pymssql://xxxx/')
age = pd.read_sql(
"""   
SELECT tract
      ,age_group.age_group_id
      ,age_group.name as age
      ,sum(population) as population
  FROM [demographic_warehouse].[fact].[age_sex_ethnicity] ase 
  JOIN demographic_warehouse.dim.mgra_denormalize md on ase.mgra_id = md.mgra_id
  JOIN demographic_warehouse.dim.age_group on ase.age_group_id = age_group.age_group_id
  WHERE datasource_id = 45 and yr_id = 2020
  GROUP BY md.tract, age_group.age_group_id, age_group.name
  ORDER BY tract, age_group_id
""",con=ddam.connect())


def age_group_lower_bound(x):
    if x =='Under 5':
        return 0
    if x == '5 to 9':
        return 5
    if x == '10 to 14':
        return 10
    if x == '15 to 17':
        return 15
    if x == '18 and 19':
        return 18
    if x == '20 to 24':
        return 20
    if x == '25 to 29':
        return 25
    if x == '30 to 34':
        return 30
    if x == '35 to 39':
        return 35
    if x == '40 to 44':
        return 40
    if x == '45 to 49':
        return 45
    if x == '50 to 54':
        return 50
    if x == '55 to 59':
        return 55
    if x == '60 and 61':
        return 60
    if x == '62 to 64':
        return 62
    if x == '65 to 69':
        return 65
    if x == '70 to 74':
        return 70
    if x == '75 to 79':
        return 75
    if x == '80 to 84':
        return 80
    if x == '85 and Older':
        return 85

age['lower_bound'] = age['age'].apply(age_group_lower_bound)


def age_group_upper_bound(x):
    if x =='Under 5':
        return 5
    if x == '5 to 9':
        return 9
    if x == '10 to 14':
        return 14
    if x == '15 to 17':
        return 17
    if x == '18 and 19':
        return 19
    if x == '20 to 24':
        return 24
    if x == '25 to 29':
        return 29
    if x == '30 to 34':
        return 34
    if x == '35 to 39':
        return 39
    if x == '40 to 44':
        return 44
    if x == '45 to 49':
        return 49
    if x == '50 to 54':
        return 54
    if x == '55 to 59':
        return 59
    if x == '60 and 61':
        return 61
    if x == '62 to 64':
        return 64
    if x == '65 to 69':
        return 69
    if x == '70 to 74':
        return 74
    if x == '75 to 79':
        return 79
    if x == '80 to 84':
        return 84
    if x == '85 and Older':
        return 85
        
age['upper_bound'] = age['age'].apply(age_group_upper_bound)



def MedianAge(Table, Geo):
    newtable = []

    for y in Table[Geo].unique():
        x = Table[Table[Geo] == y]
        
        # If the sum of pop is 0, then assign -1 as the median and append to the table
        if x['population'].sum() == 0:
            newtable.append([y, -1])
            continue
        x['cumulative_population'] = x['population'].cumsum(axis=0)
        
        # Check the dataframe
        # print(y)
        # print(x[x['cumulative_population'] > x['population'].sum()/2])

        x_median_age_class = x[x['cumulative_population'] > x['population'].sum()/2].iloc[[0]]
        x_precedingcumulative = x[x['cumulative_population'] < x['population'].sum()/2].iloc[[-1]]
        
        lower_limit = x_median_age_class.iloc[0]['lower_bound']
        width = (x_median_age_class.iloc[0]['upper_bound'] - x_median_age_class.iloc[0]['lower_bound'])
        N = x['population'].sum()
        C = x_precedingcumulative.iloc[0]['cumulative_population']
        F = x_median_age_class.iloc[0]['population']
        
        newtable.append([y, (lower_limit + (width * ((N/2 - C) / F)).round())])

    return pd.DataFrame(newtable, columns=['Geography','Median_Age'])

In [ ]:
MedianAge(age,'tract')